In [1]:
import pandas as pd

In [2]:
train=pd.read_csv('./datasets/train.csv',encoding_errors='ignore')
test=pd.read_csv('./datasets/test.csv',encoding_errors='ignore')

In [3]:
train[:5]

,uuid,title,author,abstract,Keywords,label
0,0,Accessible Visual Artworks for Blind and Visua...,"Quero, Luis Cavazos; Bartolome, Jorge Iranzo; ...",Despite the use of tactile graphics and audio ...,accessibility technology; multimodal interacti...,0
1,1,Seizure Detection and Prediction by Parallel M...,"Li, Chenqi; Lammie, Corey; Dong, Xuening; Amir...","During the past two decades, epileptic seizure...",CNN; Seizure Detection; Seizure Prediction; EE...,1
2,2,Fast ScanNet: Fast and Dense Analysis of Multi...,"Lin, Huangjing; Chen, Hao; Graham, Simon; Dou,...",Lymph node metastasis is one of the most impor...,Histopathology image analysis; computational p...,1
3,3,Long-Term Effectiveness of Antiretroviral Ther...,"Huang, Peng; Tan, Jingguang; Ma, Wenzhe; Zheng...",In order to assess the effectiveness of the Ch...,HIV; ART; mortality; observational cohort stud...,0
4,4,Real-Time Facial Affective Computing on Mobile...,"Guo, Yuanyuan; Xia, Yifan; Wang, Jing; Yu, Hui...",Convolutional Neural Networks (CNNs) have beco...,facial affective computing; convolutional neur...,0


In [4]:
train['text']=train['title'].fillna('')+' '+train['author'].fillna('')+' '+train['abstract'].fillna('')+' '+train['Keywords'].fillna('')
test['text']=test['title'].fillna('')+' '+test['author'].fillna('')+' '+test['abstract'].fillna('')+' '+test['Keywords'].fillna('')

In [5]:
from sklearn.feature_extraction.text import CountVectorizer

vec=CountVectorizer().fit(train['text'])
train_vec=vec.transform(train['text'])
test_vec=vec.transform(test['text'])

In [31]:
!pip install nltk

Looking in indexes: https://mirrors.cernet.edu.cn/pypi/web/simple


In [6]:
from nltk import word_tokenize,ngrams
#引入分词器


In [7]:
#停用词去除

stops = [
    'will', 'can', "couldn't", 'same', 'own', "needn't", 'between', "shan't", 'very',
     'so', 'over', 'in', 'have', 'the', 's', 'didn', 'few', 'should', 'of', 'that', 
     'don', 'weren', 'into', "mustn't", 'other', 'from', "she's", 'hasn', "you're",
     'ain', 'ours', 'them', 'he', 'hers', 'up', 'below', 'won', 'out', 'through',
     'than', 'this', 'who', "you've", 'on', 'how', 'more', 'being', 'any', 'no',
     'mightn', 'for', 'again', 'nor', 'there', 'him', 'was', 'y', 'too', 'now',
     'whom', 'an', 've', 'or', 'itself', 'is', 'all', "hasn't", 'been', 'themselves',
     'wouldn', 'its', 'had', "should've", 'it', "you'll", 'are', 'be', 'when', "hadn't",
     "that'll", 'what', 'while', 'above', 'such', 'we', 't', 'my', 'd', 'i', 'me',
     'at', 'after', 'am', 'against', 'further', 'just', 'isn', 'haven', 'down',
     "isn't", "wouldn't", 'some', "didn't", 'ourselves', 'their', 'theirs', 'both',
     're', 'her', 'ma', 'before', "don't", 'having', 'where', 'shouldn', 'under',
     'if', 'as', 'myself', 'needn', 'these', 'you', 'with', 'yourself', 'those',
     'each', 'herself', 'off', 'to', 'not', 'm', "it's", 'does', "weren't", "aren't",
     'were', 'aren', 'by', 'doesn', 'himself', 'wasn', "you'd", 'once', 'because', 'yours',
     'has', "mightn't", 'they', 'll', "haven't", 'but', 'couldn', 'a', 'do', 'hadn',
     "doesn't", 'your', 'she', 'yourselves', 'o', 'our', 'here', 'and', 'his', 'most',
     'about', 'shan', "wasn't", 'then', 'only', 'mustn', 'doing', 'during', 'why',
     "won't", 'until', 'did', "shouldn't", 'which'
]

In [16]:
def extract_keyword_by_freq(title,abstract):
    # 通过标题和摘要提取关键词
    #print(title.lower())
    #print(abstract.lower())
    #全转为小写
    ngrams_count=list(ngrams(word_tokenize(title.lower()),2))+list(ngrams(word_tokenize(abstract.lower()),2))
    #print(ngrams_count)
    #print("-----------")
    ngrams_count=pd.DataFrame(ngrams_count)
    #print(ngrams_count)
    #print("-----------")
    #去除存在于停用表中的二元组
    ngrams_count=ngrams_count[~ngrams_count[0].isin(stops)]
    ngrams_count=ngrams_count[~ngrams_count[1].isin(stops)]
    ngrams_count=ngrams_count[ngrams_count[0].apply(len) >3]
    ngrams_count=ngrams_count[ngrams_count[1].apply(len) >3]
    ngrams_count['phrase']=ngrams_count[0]+' '+ngrams_count[1]
    ngrams_count=ngrams_count['phrase'].value_counts()
    #计算每个二元组的出现频率
    #print(ngrams_count)
    #print("------------")
    ngrams_count=ngrams_count[ngrams_count>1]
    return list(ngrams_count.index)[:5]

In [22]:
train['abstract']=train['abstract'].fillna('')
train['title']=train['title'].fillna('')
test['abstract']=test['abstract'].fillna('')
test['title']=test['title'].fillna('')


In [27]:
from sklearn.linear_model import LogisticRegression
# 过滤警告消息
from warnings import simplefilter
from sklearn.exceptions import ConvergenceWarning
simplefilter("ignore", category=ConvergenceWarning)

md=LogisticRegression()
md.fit(train_vec,train['label'])
test['label']=md.predict(test_vec)

In [24]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [29]:
test_words=[]

In [30]:
for row in test.iterrows():
    prediction_keywords=extract_keyword_by_freq(row[1].title,row[1].abstract)
    prediction_keywords=[x.title() for x in prediction_keywords]#首字母转为大写
    if len(prediction_keywords)==0:
        prediction_keywords=['A','B']
    test_words.append(';'.join(prediction_keywords))
test['Keywords']=test_words
test[['uuid','Keywords','label']].to_csv('submit_task2.csv',index=None)